# Submission

In [365]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [366]:
submission_with_whole_train = True

# Feature engineer (option 1)

In [367]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
if not submission_with_whole_train:
    stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [368]:
from utils import split_plaace_cat


store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    }

if not submission_with_whole_train:
    store_dataframes["val"] = stores_val

for df_name, df in store_dataframes.items():
    df = split_plaace_cat(df)

In [369]:
store_dataframes["train"].shape

(12859, 16)

In [384]:
store_dataframes["train"]["log_revenue"] = store_dataframes["train"].revenue.apply(lambda x: np.log1p(x))
if not submission_with_whole_train:
    store_dataframes["val"]["log_revenue"] = store_dataframes["val"].revenue.apply(lambda x: np.log1p(x))

In [370]:
from utils import create_geographical_columns, create_chain_and_mall_columns, generate_rev_dict, generate_chain_rev_dict, create_mean_chain_rev_col

chain_count = stores_train["chain_name"].value_counts().to_dict()
lower_limit = 1
rev_dict, mean_revenue = generate_rev_dict(store_dataframes["train"])

for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_geographical_columns(df)
    store_dataframes[df_name] = create_chain_and_mall_columns(df, chain_count, lower_limit=lower_limit)

chain_rev_dict = generate_chain_rev_dict(store_dataframes["train"])
for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_mean_chain_rev_col(df, bounded_chain_revs=chain_rev_dict)


100%|██████████| 3/3 [00:00<00:00, 92.21it/s]


In [373]:
from utils import concat_df_keep_unq_index
concat_df = concat_df_keep_unq_index(stores_train, stores_extra)

!NB next cell takes 3 minutes to run (if comp_plaace_gran. = [1, 2, 3, 4]) (on M1 Mac with 16GB RAM)

(outdated) If you have the .csv files temp_data/closest_comp_\{df_name\}, skip running this cell and run the cell below it instead

In [374]:
from utils import find_dist_to_nearest_comp

nearest_comp_plaace_cat_gran = [1, 2, 3, 4]
n_nearest_comp = [1, 2, 3, 4, 5, 7, 10]

store_dataframes["train"] = find_dist_to_nearest_comp(
    store_dataframes["train"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [375]:
if not submission_with_whole_train:
    store_dataframes["val"] = find_dist_to_nearest_comp(
        store_dataframes["val"], 
        nearest_comp_plaace_cat_gran, 
        n_nearest_comp, 
        training=True, 
        training_df=concat_df,
        )

In [376]:
store_dataframes["test"] = find_dist_to_nearest_comp(
    store_dataframes["test"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=False,
    training_df=concat_df
)

In [381]:
comp_plaace_cols = list(store_dataframes["train"].columns[-56:])

In [385]:
dist_dict = store_dataframes["train"][['log_revenue'] + comp_plaace_cols].corr().iloc[0].to_dict()
sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['sum_dist_to_nearest_10_comp_plaace_1', -0.1105480705544587],
 ['mean_dist_to_nearest_10_comp_plaace_1', -0.1105480705544587],
 ['sum_dist_to_nearest_7_comp_plaace_1', -0.10622134586388843],
 ['mean_dist_to_nearest_7_comp_plaace_1', -0.10622134586388843],
 ['sum_dist_to_nearest_5_comp_plaace_1', -0.10081384666797813],
 ['mean_dist_to_nearest_5_comp_plaace_1', -0.10081384666797813],
 ['sum_dist_to_nearest_4_comp_plaace_1', -0.09573147190735849],
 ['mean_dist_to_nearest_4_comp_plaace_1', -0.09573147190735849],
 ['sum_dist_to_nearest_3_comp_plaace_1', -0.08901731863544961],
 ['mean_dist_to_nearest_3_comp_plaace_1', -0.08901731863544961],
 ['sum_dist_to_nearest_2_comp_plaace_1', -0.08312262382854094],
 ['mean_dist_to_nearest_2_comp_plaace_1', -0.08312262382854094],
 ['sum_dist_to_nearest_1_comp_plaace_1', -0.07028366698451731],
 ['mean_dist_to_nearest_1_comp_plaace_1', -0.07028366698451731],
 ['sum_dist_to_nearest_2_comp_plaace_2', -0.0512674190289544],
 ['mean_dis

In [387]:
comp_relevant_cols = sorted_relevant_dist_cols[1:14:2]
comp_relevant_cols = [r[0] for r in comp_relevant_cols]

In [388]:
comp_relevant_cols

['sum_dist_to_nearest_10_comp_plaace_1',
 'sum_dist_to_nearest_7_comp_plaace_1',
 'sum_dist_to_nearest_5_comp_plaace_1',
 'sum_dist_to_nearest_4_comp_plaace_1',
 'sum_dist_to_nearest_3_comp_plaace_1',
 'sum_dist_to_nearest_2_comp_plaace_1',
 'sum_dist_to_nearest_1_comp_plaace_1']

!NB next cell takes 15 minutes to run. Grab a coffee or somtething while you wait :) (on M1 Mac with 16GB RAM)

If you have the .csv files temp_data/closest_bus_stop_\{df_name\}, skip running this cell and run the cell below it instead

In [389]:
from bus_utils import find_closest_bus_stop

bus_stop_n = [1, 3, 5, 7]
bus_mean = True
bus_sum = True

bus_stop_columns = []

if(bus_sum):
    bus_stop_columns += [f"closest_bus_stop_sum_{i}" for i in bus_stop_n]

if(bus_mean):
    bus_stop_columns += [f"closest_bus_stop_mean_{i}" for i in bus_stop_n]


for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = find_closest_bus_stop(df, bus_stop_n, _sum=bus_sum, _mean=bus_mean)    

100%|██████████| 3/3 [12:15<00:00, 245.27s/it]


In [390]:
bus_dict = store_dataframes["train"][['log_revenue'] + bus_stop_columns].corr().iloc[0].to_dict()
bus_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(bus_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
bus_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['closest_bus_stop_sum_7', -0.04147195329213889],
 ['closest_bus_stop_mean_7', -0.04147195329213841],
 ['closest_bus_stop_sum_5', -0.03935581657499084],
 ['closest_bus_stop_mean_5', -0.039355816574990564],
 ['closest_bus_stop_sum_3', -0.03794766139621982],
 ['closest_bus_stop_mean_3', -0.03794766139621965],
 ['closest_bus_stop_sum_1', -0.03295366682075893],
 ['closest_bus_stop_mean_1', -0.03295366682075893]]

In [393]:
bus_relevant_cols = bus_sorted_relevant_dist_cols[1:9:2]
bus_relevant_cols = [r[0] for r in bus_relevant_cols]

In [394]:
bus_relevant_cols

['closest_bus_stop_sum_7',
 'closest_bus_stop_sum_5',
 'closest_bus_stop_sum_3',
 'closest_bus_stop_sum_1']

In [396]:
from grunnkrets import make_grunnkrets_df

full_population_dataframes = {}
full_pop_columns = []

for df_name, df in tqdm(store_dataframes.items()):
    full_population_dataframes[df_name] = make_grunnkrets_df(df)
    full_pop_columns = full_population_dataframes[df_name].columns

100%|██████████| 3/3 [00:25<00:00,  8.39s/it]


In [412]:
full_pop_columns = list(full_pop_columns[-184:])

In [410]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = df.merge(
        full_population_dataframes[df_name], 
        left_index=True,
        right_index=True,
        how="outer", 
        suffixes=('', '_redundant')
    )
    store_dataframes[df_name].drop(store_dataframes[df_name].filter(regex='_redundant$').columns, axis=1, inplace=True)

In [418]:
full_pop_dict = store_dataframes["train"][['log_revenue'] + full_pop_columns].corr().iloc[0].to_dict()
full_pop_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(full_pop_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
full_pop_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['delomrade.mean_age', -0.03629222923756194],
 ['delomrade.c_age_56-90_ratio', -0.035473331078469346],
 ['kommune.c_age_19-30_ratio', 0.035367899040518964],
 ['delomrade.c_age_19-30_ratio', 0.03291331707473177],
 ['grunnkrets_id.pop_density', -0.03171235964527871],
 ['grunnkrets_id.c_age_19-30_ratio', 0.031291477649653884],
 ['grunnkrets_id.income_density', -0.028689875454496534],
 ['kommune.mean_age', -0.028416803074053056],
 ['grunnkrets_id.income_density_log', -0.02814778437192616],
 ['grunnkrets_id.pop_density_log', -0.026520651875512453],
 ['kommune.c_age_56-90_ratio', -0.02641575812019863],
 ['grunnkrets_id.tot_pop_log', 0.021469316783325483],
 ['fylke.couple_with_children_income', -0.019571321675668447],
 ['grunnkrets_id.total_income_log', 0.019371702140288372],
 ['kommune.total_income_log', 0.019130375499943404],
 ['kommune.area_km2', 0.019096522730079477],
 ['kommune.tot_pop_log', 0.01891266184858848],
 ['grunnkrets_id.c_age_0-18_ratio', -0.017748643890

In [425]:
full_pop_relevant_cols = full_pop_sorted_relevant_dist_cols[1:8]
full_pop_relevant_cols = [r[0] for r in full_pop_relevant_cols]

In [426]:
full_pop_relevant_cols

['delomrade.mean_age',
 'delomrade.c_age_56-90_ratio',
 'kommune.c_age_19-30_ratio',
 'delomrade.c_age_19-30_ratio',
 'grunnkrets_id.pop_density',
 'grunnkrets_id.c_age_19-30_ratio',
 'grunnkrets_id.income_density']

In [427]:
fylke_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("grunnkrets_id.")]

In [428]:
from utils import mean_func_rev, generate_rev_dict

for i in range(1, 5):
    rev_dict, mean_revenue = generate_rev_dict(stores_train, i)
    for df_name, df in store_dataframes.items():
        df["mean_revenue_" + str(i)] = df["plaace_cat_" + str(i)].apply(lambda x: mean_func_rev(x, rev_dict, mean_revenue))
        df["log_mean_revenue_" + str(i)] = df["mean_revenue_" + str(i)].apply(lambda x: np.log1p(x))

In [429]:
from num_stores import add_num_stores_info

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_num_stores_info(df)

/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns wh

In [432]:
num_store_cols = list(store_dataframes["train"].columns[-64:])

In [433]:
num_store_dict = store_dataframes["train"][['log_revenue'] + num_store_cols].corr().iloc[0].to_dict()
num_store_sorted_relevant_cols = [[k, v] for k, v in sorted(num_store_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
num_store_sorted_relevant_cols

[['log_revenue', 1.0],
 ['fylke.plaace_cat_1_per_capita', 0.3019810012107429],
 ['fylke.plaace_cat_1_per_tot_income', 0.29693232307328526],
 ['kommune.plaace_cat_1_per_capita', 0.2516145481685565],
 ['kommune.plaace_cat_1_per_tot_income', 0.24004506651234722],
 ['fylke.plaace_cat_1_count', 0.18277282826147828],
 ['fylke.plaace_cat_3_per_capita', 0.17406152041570094],
 ['fylke.plaace_cat_3_per_tot_income', 0.17021599679509367],
 ['fylke.plaace_cat_4_per_capita', 0.15697350330354706],
 ['fylke.plaace_cat_4_per_tot_income', 0.15581052126602077],
 ['grunnkrets_id.plaace_cat_1_count', 0.12069436266715614],
 ['delomrade.plaace_cat_1_count', 0.10653236446339913],
 ['fylke.plaace_cat_3_count', 0.0968517411856443],
 ['kommune.plaace_cat_3_per_capita', 0.0857966291167957],
 ['kommune.plaace_cat_1_per_km2', 0.0807174589750661],
 ['fylke.plaace_cat_4_count', 0.07748725348442329],
 ['kommune.plaace_cat_3_per_tot_income', 0.07553795763283432],
 ['grunnkrets_id.plaace_cat_1_per_capita', 0.06857860468

In [437]:
num_store_relevant_cols = num_store_sorted_relevant_cols[1:15]
num_store_relevant_cols = [r[0] for r in num_store_relevant_cols]

In [438]:
num_store_relevant_cols

['fylke.plaace_cat_1_per_capita',
 'fylke.plaace_cat_1_per_tot_income',
 'kommune.plaace_cat_1_per_capita',
 'kommune.plaace_cat_1_per_tot_income',
 'fylke.plaace_cat_1_count',
 'fylke.plaace_cat_3_per_capita',
 'fylke.plaace_cat_3_per_tot_income',
 'fylke.plaace_cat_4_per_capita',
 'fylke.plaace_cat_4_per_tot_income',
 'grunnkrets_id.plaace_cat_1_count',
 'delomrade.plaace_cat_1_count',
 'fylke.plaace_cat_3_count',
 'kommune.plaace_cat_3_per_capita',
 'kommune.plaace_cat_1_per_km2']

# Removing outliers

In [439]:
from outlier_utils import remove_low_revenue

store_dataframes["train"] = remove_low_revenue(store_dataframes["train"], low_rev_limit=0.05)

# Read data directly from CSV (option 2)

In [356]:
stores_train = pd.read_csv("temp_data/full_features_train.csv", index_col=0)
stores_val = pd.read_csv("temp_data/full_features_val.csv", index_col=0)
stores_extra = pd.read_csv("temp_data/full_features_extra.csv", index_col=0)
stores_test = pd.read_csv("temp_data/full_features_test.csv", index_col=0)

store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    "val": stores_val
    }

In [31]:
with open("relevant_cols.txt", "r") as f:
    data = f.readlines()

comp_relevant_cols = data[0].strip().split(",")
bus_relevant_cols = data[1].strip().split(",")
num_stores_relevant_cols = data[2].strip().split(",")

In [102]:
with open("relevant_features.txt", "r") as f:
    fw_relevant_cols = f.read().strip().split(";")

In [123]:
fw_OH_cols = []
fw_OE_cols = []
fw_num_cols = []
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(0))
fw_OE_cols.append(fw_relevant_cols.pop(6))
fw_num_cols = fw_relevant_cols

# Training the model

In [440]:
from RMSLE import rmsle
from pred_var_utils import reverse_log1p_transform_pred_var

In [462]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_2", "plaace_cat_3", "plaace_cat_4"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = [] #["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", 
#"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
"log_mean_revenue_1", "log_mean_revenue_2", "log_mean_revenue_3", "log_mean_revenue_4", 
"log_chain_mean_revenue"
] + full_pop_relevant_cols + comp_relevant_cols + bus_relevant_cols + num_store_relevant_cols
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(store_dataframes["train"])
#X_val = preprocessor.transform(store_dataframes["val"])

In [463]:
y_train = np.array(store_dataframes["train"].log_revenue)
if not submission_with_whole_train:
    y_val = np.array(store_dataframes["val"].revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

## Apply PCA 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(.95)

pca.fit(stores_train)

pca.n_components_


ValueError: could not convert string to float: '990857423-990974489-88185'

## Random Forest

In [449]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=250, max_features=8, min_samples_leaf=2, min_samples_split=16)

In [450]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_features=8, min_samples_leaf=2, min_samples_split=16,
                      n_estimators=250, n_jobs=-1, random_state=0)

In [451]:
rf_y_pred = reverse_log1p_transform_pred_var(rf.predict(X_val), std_y=std_y, mean_y=mean_y)


In [456]:
rmsle(y_val, rf_y_pred)

AssertionError: y_true and y_pred have different shapes

## Linear Regression

In [452]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs=-1)

In [453]:
lr.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [454]:
lr_y_pred = reverse_log1p_transform_pred_var(lr.predict(X_val), std_y, mean_y)
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [455]:
rmsle(y_val, lr_y_pred)

AssertionError: y_true and y_pred have different shapes

## Light GBM

In [338]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=0, n_jobs=-1, learning_rate=0.1, n_estimators=100, reg_lambda=0.01)

In [339]:
lgbm.fit(X_train, y_train)

LGBMRegressor(random_state=0, reg_lambda=0.01)

In [340]:
lgbm_y_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_val), std_y, mean_y)
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [341]:
rmsle(y_val, lgbm_y_pred)

0.725126439550935

## CatBoost

In [464]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

cb = CatBoostRegressor(
    random_seed=0, 
    verbose=False, 
    depth= 4, 
    l2_leaf_reg= 5, 
    learning_rate= 0.1
    )

cb_params = grid = {
    'learning_rate': [0.05, 0.1, 0.3],
    'depth': [2, 3, 4, 5, 6],
    'l2_leaf_reg': [4, 5], 
    }

#cb_clf = cb.grid_search(cb_params, X=X_train, y=y_train)

In [343]:
cb_clf["params"]

NameError: name 'cb_clf' is not defined

In [465]:
cb.fit(X_train, y_train)

In [345]:
cb_y_pred = reverse_log1p_transform_pred_var(cb.predict(X_val), std_y, mean_y)
cb_y_pred = np.array([max(0, xi) for xi in cb_y_pred])

In [346]:
rmsle(y_val, cb_y_pred)

0.7209299235429011

## Stacking classifiers

In [347]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('cb', cb), 
    ('lgbm', lgbm), 
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=CatBoostRegressor(random_state=42, verbose=False)
)

In [348]:
reg.fit(X_train, y_train)

StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(max_features=8,
                                                     min_samples_leaf=2,
                                                     min_samples_split=16,
                                                     n_estimators=250,
                                                     n_jobs=-1,
                                                     random_state=0)),
                              ('cb',
                               <catboost.core.CatBoostRegressor object at 0x2bd929e20>),
                              ('lgbm',
                               LGBMRegressor(random_state=0, reg_lambda=0.01))],
                  final_estimator=<catboost.core.CatBoostRegressor object at 0x2bd929eb0>)

In [349]:
reg_y_pred = reverse_log1p_transform_pred_var(reg.predict(X_val), std_y, mean_y)

In [350]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

0.7327124215229028

## Creating the submission

In [466]:
# Predict on the test set 
X_test = preprocessor.transform(store_dataframes["test"])
y_test_pred = reverse_log1p_transform_pred_var(cb.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)

# Create CSV of dataframes

In [101]:
for df_name, df in store_dataframes.items():
    filepath = Path(f"temp_data/full_features_{df_name}.csv")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, index=True)